In [1]:
import evebox as box
import pandas as pd
import tensorflow as tf
from tqdm.notebook import tqdm, trange

In [2]:
uni = box.Universe.from_esi(cache = 'universe.txt', tqdm = tqdm).range('Jita', min_sec = 0.9).with_market_types()

In [3]:
#orders = box.load_orders(uni, tqdm = tqdm)
orders = pd.read_csv('market/current.csv')

In [4]:
#orders.to_csv('market/current.csv')

In [5]:
orders

,order_id,duration,is_buy_order,issued,location_id,min_volume,price,range,system_id,type_id,volume_remain,volume_total
0,911203062,365,False,2020-01-02T11:04:27+00:00,60000298,1,50000.00,region,30000129,967,1,1
1,911203074,365,False,2020-03-08T14:36:26+00:00,60000298,1,109.02,region,30000129,3673,94161,94161
2,911203098,365,True,2020-03-08T14:25:05+00:00,60000298,1,208.99,station,30000129,3717,13606,13606
3,911203190,365,False,2020-01-23T11:09:24+00:00,60000358,1,50000.00,region,30000144,967,1,1
4,911203191,365,False,2020-02-03T11:06:18+00:00,60000361,1,50000.00,region,30000142,967,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
258434,5628621294,14,False,2020-03-08T15:09:36+00:00,60015027,1,9.00,region,30030141,34,7250,7250
258435,5628621295,90,True,2020-03-08T15:09:37+00:00,1028858195912,1,1830000.09,1,30000144,2195,20,20
258436,5628621307,30,False,2020-03-08T15:09:38+00:00,60003760,1,99888.98,region,30000142,20420,2000,2000
258437,5628621309,90,False,2020-03-08T15:09:38+00:00,60003760,1,1399.79,region,30000142,27447,100000,100000


In [6]:
gym = box.tf.TradingGym(uni, orders)

In [13]:
jita = [s["system_id"] for s in uni.systems.values() if s["name"] == "Jita"][0]

state = box.MutableState()
state.time_left = 100.0
state.system    = jita

state = box.State(state)

In [19]:
model = box.tf.ReferenceModel(uni, orders, state_shape = [64])

unroller = gym.unroll_model(model, tqdm)

In [25]:
with tf.GradientTape() as tape:
    result = unroller(
        state, 150
    )

In [26]:
result[0:10]

[(
  State:
      Validation: Valid
      
      System:     30000142 (30000142)
      Station ID: None
      
      Resources:
          Time:           100.00
          Wallet:           0.00
          Collateral:       0.00 /       0.00 (      0.00 used)
          Cargo:            0.00 /       0.00 (      0.00 used)
          
      Cargo contents:
          
          
      Modified orders:
          Empty DataFrame
  Columns: [ID, Remaining]
  Index: []
          ,
  Move to system 30000143,
  <tf.Tensor: shape=(), dtype=float32, numpy=-13.748633>,
  <tf.Tensor: shape=(), dtype=float32, numpy=0.0>),
 (
  State:
      Validation: Valid
      
      System:     30000143 (30000143)
      Station ID: None
      
      Resources:
          Time:            99.00
          Wallet:           0.00
          Collateral:       0.00 /       0.00 (      0.00 used)
          Cargo:            0.00 /       0.00 (      0.00 used)
          
      Cargo contents:
          
          
      Mod

In [10]:
tape.gradient(result[0][2], sources=model.trainable_variables)

[<tf.Tensor: shape=(), dtype=float32, numpy=-5.9604645e-08>]

In [11]:
model.trainable_variables

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>]